# Some thougths on autograd and gradient calculations

Set Up

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
torch.set_default_dtype(torch.float64)

if torch.cuda.is_available():
    device = torch.device('cuda')
    dtype = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')
    dtype = torch.FloatTensor

# If you don't want to bother with the device, stay on cpu:
# device = torch.device('cpu')

print(f"Using {device}")

!nvidia-smi

# set seeds for reproductibility
random_seed = 42
rng = np.random.default_rng(seed=random_seed)
torch.manual_seed(random_seed)

Using cpu
/bin/bash: line 1: nvidia-smi: command not found


# Backward

In [6]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

print(f"a : {a.size()}")
print(f"b : {b.size()}")

a : torch.Size([2])
b : torch.Size([2])


In [7]:
q = 3*a**3 + 2*b**2

print(f"q = {q}")
print(f"q : {q.size()}")

q = tensor([ 96., 113.], grad_fn=<AddBackward0>)
q : torch.Size([2])


ici :
outputs = q ; size = 2
inputs = (a, b) les "leafs", size = 2
calcul de la Jacobienne 2 x 2

grad_outputs = 2 x 1

torch.autograd.grad => $J^{T} . \text{grad-outputs}$ : 2 x 1

In [9]:
external_grad = torch.tensor([1., 1.])

In [10]:
q.backward(gradient=external_grad)

$\nabla_a q = 9 * a^2$

In [12]:
a.grad

tensor([36., 81.])

$\nabla_b q = 4*b$

In [14]:
b.grad

tensor([24., 16.])

# torch.autograd.grad

torch.autograd.grad(

outputs = le tensor que l'on veut différencier
    
inputs = le tensor par rapport auquel on va différencier outputs
    
grad_outputs = le vecteur de même dimension que outputs, intervient dans le calcul de $J^{T} . v$

)

In [17]:
n_samples = 5

In [18]:
X = torch.randn((n_samples), requires_grad=True)
W = torch.tensor([2.0], requires_grad=True)

print(f"X : {X.size()}")
print(f"W : {W.size()}")

X : torch.Size([5])
W : torch.Size([1])


In [19]:
Y = W * X
Y.size()

torch.Size([5])

### torch.autograd.grad (outputs=Y, inputs=W, grad_outputs=v) va calculer:

$(\nabla_W Y)^T.v$ shape : (1 x n_samples) x (n_samples x 1) = 1

In [21]:
external_grad = torch.ones(Y.size()[0])
external_grad.size()

torch.Size([5])

In [22]:
g = torch.autograd.grad(outputs=Y, inputs=W, grad_outputs=external_grad, retain_graph=True)

In [23]:
g[0].size()

torch.Size([1])

In [24]:
g[0]

tensor([-0.1926])

$(\nabla_W Y)^T.v = \sum_{j=1}^{n} \frac{\partial{Y}_i}{\partial W} = \sum_{j=1}^{n} X_i$

In [26]:
X.sum()

tensor(-0.1926, grad_fn=<SumBackward0>)

### torch.autograd.grad (outputs=Y, inputs=X, grad_outputs=v) va calculer:

$(\nabla_X Y)^T.v$ shape : (n_samples x n_samples) x (n_samples x 1) = n_samples

In [28]:
external_grad = torch.ones(Y.size()[0])
external_grad.size()

torch.Size([5])

In [29]:
h = torch.autograd.grad(outputs=Y, inputs=X, grad_outputs=external_grad)

In [30]:
h[0].size()

torch.Size([5])

In [31]:
h[0]

tensor([2., 2., 2., 2., 2.])

$(\nabla_X Y)^T.v = \frac{\partial{Y}_i}{\partial{X}_j} @ v_k = W_k = 2.0$

### Gradient d'une fonction scalaire de plusieurs variables

In [34]:
n_samples = 5
n_dim = 3

X = torch.randn((n_samples, n_dim), requires_grad=True)

In [35]:
X

tensor([[-0.1863,  2.2082, -0.6380],
        [ 0.4617,  0.2674,  0.5349],
        [ 0.8094,  1.1103, -1.6898],
        [-0.9890,  0.9580,  1.3221],
        [ 0.8172, -0.7658, -0.7506]], requires_grad=True)

In [36]:
W = torch.tensor([1.0,2.0,3.0])

In [37]:
Y = X @ W
print(Y)

tensor([ 2.3161,  2.6011, -2.0395,  4.8934, -2.9664], grad_fn=<MvBackward0>)


In [38]:
external_gradients = torch.ones(Y.size()[0])
external_gradients

tensor([1., 1., 1., 1., 1.])

torch.autograd.grad (outputs=Y, inputs=X, grad_outputs=external_gradients) va calculer :

$(\nabla_X Y)^T . v \sim (\nabla_X Y_1, \nabla_X Y_2, ..., \nabla_X Y_n) . (v_1, v_2, ... v_n) = (\nabla_X Y_1.v_1, \nabla_X Y_2.v_2, ..., \nabla_X Y_n.v_n)$ shape : (n_samples x n_dim)

In [40]:
j = torch.autograd.grad(outputs=Y, inputs=X, grad_outputs=external_gradients,retain_graph=True)

In [41]:
j[0]

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.]])

In [48]:
external_gradients=torch.tensor([0.0,1.0,2.0,3.0,-1.0])

In [50]:
k = torch.autograd.grad(outputs=Y, inputs=X, grad_outputs=external_gradients)

In [52]:
k[0]

tensor([[ 0.,  0.,  0.],
        [ 1.,  2.,  3.],
        [ 2.,  4.,  6.],
        [ 3.,  6.,  9.],
        [-1., -2., -3.]])